In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/ChristmasCarol/character-tagging/ChristmasCarol.entities', delimiter='\t')
df

,COREF,start_token,end_token,prop,cat,text
0,14,0,3,PROP,PER,MARLEY 'S GHOST Marley
1,75,22,22,PRON,PER,his
2,76,27,28,NOM,PER,the clergyman
3,77,30,31,NOM,PER,the clerk
4,78,33,34,NOM,PER,the undertaker
...,...,...,...,...,...,...
4737,1155,35930,35930,PRON,PER,us
4738,58,35936,35937,PROP,PER,Tiny Tim
4739,18,35940,35940,PROP,PER,God
4740,1155,35942,35942,PRON,PER,Us


In [3]:
df_characters = df[df.cat == 'PER']
df_characters

,COREF,start_token,end_token,prop,cat,text
0,14,0,3,PROP,PER,MARLEY 'S GHOST Marley
1,75,22,22,PRON,PER,his
2,76,27,28,NOM,PER,the clergyman
3,77,30,31,NOM,PER,the clerk
4,78,33,34,NOM,PER,the undertaker
...,...,...,...,...,...,...
4737,1155,35930,35930,PRON,PER,us
4738,58,35936,35937,PROP,PER,Tiny Tim
4739,18,35940,35940,PROP,PER,God
4740,1155,35942,35942,PRON,PER,Us


In [4]:
len(df_characters.groupby(df.COREF))

797

In [5]:
characters = {}

for coref_id, coref_df in df_characters.groupby(df.COREF):
    try:
        proper_noun_df = coref_df[coref_df.prop == 'PROP']
        if not proper_noun_df.empty:
            name = proper_noun_df.text[proper_noun_df.text.str.len() == proper_noun_df.text.str.len().max()].unique()[0]
            characters[coref_id] = name
    except:
        continue

characters

{14: "MARLEY 'S GHOST Marley",
 16: 'Old Jacob Marley',
 17: 'Hamlet',
 18: 'God',
 19: 'Christmas',
 20: 'Mr. Ebenezer Scrooge',
 21: 'Mr. Marley',
 22: 'Abundance',
 23: 'the good St. Dunstan',
 24: 'Cains',
 25: 'Abels',
 26: 'Pharaoh',
 27: 'Queens of Sheba',
 28: 'Abrahams',
 29: 'Belshazzars',
 30: 'Apostles',
 31: 'Humbug',
 32: "Much!'--Marley",
 33: 'Thankee',
 34: 'Good Spirit',
 35: 'dear old honest Ali Baba',
 36: 'Valentine',
 37: 'Orson',
 38: 'the Genii',
 39: 'Poor Robin Crusoe',
 40: 'the Parrot',
 41: 'Friday',
 42: 'Father',
 43: 'Mrs. Fezziwig',
 44: 'Old Fezziwig',
 45: 'Dick Wilkins',
 46: 'Poor Dick',
 47: 'Jack Robinson',
 48: 'In',
 49: 'Sir Roger de Coverley',
 50: 'Belle',
 51: 'One',
 52: 'Plenty',
 53: 'Norfolk Biffins',
 54: 'Bob Cratchit',
 55: 'Mrs. Cratchit',
 56: 'Belinda Cratchit',
 57: 'Master Peter Cratchit',
 58: 'poor Tiny Tim',
 59: "Martha warn't",
 60: 'Miss Belinda',
 61: 'the two young Cratchits',
 62: 'Robert',
 63: 'Fred',
 64: 'Topper',
 6

In [6]:
from collections import defaultdict
import string


merged_idxs = set()
merged_coref_ids = defaultdict(list)

female_titles = [
    'Mrs.',
    'Lady',
    'Miss',
    'Aunt',
    'Duchess',
]

male_titles = [
    'Sir',
    'Mr.',
    'Lord',
    'Uncle',
    'Colonel',
]

neutral_titles_and_starters = [
    'Dr.',
    'Dear',
    'Poor',
    'Old',
    'Dearsest',
]


# Merge characters based on titles
for i, (coref_id, base_name) in enumerate(characters.items()):
    
    if i in merged_idxs:
        continue
    
    try:
        base_split_name = base_name.split()

        base_titles = []
        base_gender = 'neutral'
        while base_split_name[0] in male_titles + female_titles + neutral_titles_and_starters:

            if base_split_name[0] in male_titles:
                base_gender = 'male'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            elif base_split_name[0] in female_titles:
                base_gender = 'female'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            else:
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            
        base_forename = base_split_name[0]
        base_surname = base_split_name[-1]
            

        for j, (other_coref_id, other_name) in enumerate(characters.items()):
            
            if j in merged_idxs:
                continue
            
            if j <= i:
                continue
            other_split_name = other_name.split()
            
            other_titles = []
            other_gender = 'neutral'
            while any([name in male_titles + female_titles + neutral_titles_and_starters for name in other_split_name]):

                if other_split_name[0] in male_titles:
                    other_gender = 'male'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                elif other_split_name[0] in female_titles:
                    other_gender = 'female'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                else:
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])

            other_forename = other_split_name[0]
            other_surname = other_split_name[-1]
            
            if base_gender == 'male' and other_gender == 'female' or base_gender == 'female' and other_gender == 'male':
                continue
            
            if other_forename != other_surname and base_forename != base_surname:
                if base_forename != other_forename or base_surname != other_surname:
                    continue
                else:
                    merged_idxs.add(j)
                    merged_coref_ids[coref_id].append(other_coref_id)
                    print(f'Merging1: {base_name} -> {other_name}')
                    continue
            
            # Otherwise gender is the same/neutral, if the forenames match we should merge
            
            if other_forename == base_forename:
                merged_idxs.add(j)
                merged_coref_ids[coref_id].append(other_coref_id)
                print(f'Merging2: {base_name} -> {other_name}')
    except Exception as e:
        pass

Merging2: Mrs. Fezziwig -> Old Fezziwig
Merging2: Dick Wilkins -> Poor Dick
Merging2: Belinda Cratchit -> Miss Belinda


In [7]:
df_merged = df_characters[df_characters['COREF'].isin(list(characters.keys()))]
df_merged.COREF.unique().size

62

In [8]:
for coref_id, merged_coref_list in merged_coref_ids.items():
    for merged_coref_id in merged_coref_list:
        df_merged.loc[df_merged["COREF"] == merged_coref_id, "COREF"] = coref_id
    
df_merged.COREF.unique().size

59

In [9]:
min_mentions = 10

df_filtered = df_merged.groupby('COREF').filter(lambda x: len(x) > min_mentions)
df_filtered.COREF.unique().size

15

In [10]:
df_filtered

,COREF,start_token,end_token,prop,cat,text
0,14,0,3,PROP,PER,MARLEY 'S GHOST Marley
6,20,41,41,PROP,PER,Scrooge
7,20,46,46,PROP,PER,Scrooge
8,20,56,56,PRON,PER,he
9,20,60,60,PRON,PER,his
...,...,...,...,...,...,...
4729,58,35876,35876,PRON,PER,He
4730,66,35882,35882,PROP,PER,Spirits
4731,58,35901,35901,PRON,PER,him
4732,58,35903,35903,PRON,PER,he


In [11]:
df_filtered['character_name'] = df_filtered.COREF.map(lambda x: characters[x])

In [12]:
df_filtered.to_csv('../data/ChristmasCarol/processed_characters.csv')

In [13]:
import json


merged_coref_to_new_coref = {}

for coref_id, merged_list in merged_coref_ids.items():
    for merged_id in merged_coref_list:
        merged_coref_to_new_coref[merged_id] = coref_id

with open('../data/ChristmasCarol/coref_merge_map.json', 'w') as f:
    json.dump(merged_coref_to_new_coref, f)